In [1]:
import sys
import json
sys.path.append('..')

from lre.data import Relation, RelationSample, Sequence
import lre.functional as functional
import lre.models as models
import lre.metrics as metrics
import lre.logging_utils as logging_utils
from collections import defaultdict
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import torch.nn as nn
import random
import time
import logging
import copy
import os

/home/exia/miniconda3/envs/my-lre/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = 'cuda:0'
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf",\
                                             torch_dtype=torch.float16, low_cpu_mem_usage=True)
model.to(device)
tokenizer.pad_token = tokenizer.eos_token
mt = models.ModelAndTokenizer(model,tokenizer)

Loading checkpoint shards: 100%|███████████████████████████████████████████| 2/2 [00:00<00:00,  5.90it/s]


In [4]:
mt

ModelAndTokenizer(model=LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [ ]:
import os
os.listdir('llama_8_31_approx')

In [3]:
type(mt.model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [11]:
import llra.build as build
from importlib import reload
reload(functional)
reload(models)
reload(build)

<module 'llra.build' from '/home/exia/my-lre/data/../llra/build.py'>

In [14]:
start, end = 8,31
WEIGHT_NAME = f's_o_weight_{start}_{end}'
BIAS_NAME = f's_o_bias_{start}_{end}'
json_path = 'json/enckno/E06 [animal - youth].json'
file = open(json_path, 'r')
data = json.load(file)
file.close()

import llra.build as build
from baukit.baukit import parameter_names, get_parameter
import torch.nn as nn
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



def find_exact_match(filename, search_string):
    with open(filename, 'r') as file:
        for line in file:
            if search_string.lower() in line.strip().lower():
                return line.strip()
    print(f"Could not find {search_string}")
    return None

N_ICL = 8 
N_AVG = 8
N_TRIALS = 8
VIEW_SAMPLES = 5

APPROX_FOLDER = 'llama_8_31_approx'
relations = os.listdir(f'{APPROX_FOLDER}')

for beta in [1,7,14,21]:
    relations = ['meronyms - part']
    for relation_name in relations:
        #LOAD RELATION
        json_path = find_exact_match("json_paths.txt", relation_name)
        file = open(json_path, 'r')
        data = json.load(file)
        file.close()
        relation = Relation.from_dict(data)
        prompt_template = relation.prompt_templates[0]

        #LOAD WEIGHTS
        wdir = f"{APPROX_FOLDER}/{relation_name}"
        ignored_names = ['.ipynb_checkpoints', 'disappoint']
        j_samples = [x for x in os.listdir(wdir) if x not in ignored_names]
        j_samples = random.sample(j_samples, N_AVG)
        print(f'using {N_AVG} samples out of {len(j_samples)} to build LRE')
        reg_weight = build.mean_weight_or_bias(wdir,
                                               WEIGHT_NAME, j_samples).half().to(device)
        reg_bias = build.mean_weight_or_bias(wdir,
                                             BIAS_NAME, j_samples).half().to(device)
        
        #ASSEMBLE PROMPTS AND OBJECT ANSWERS
        clozed_prompts = []
        clozed_answers = []
        #Do not use the training examples for ICL or testing purposes.
        test_samples = [x for x in relation.samples if x not in j_samples]
        random.shuffle(test_samples)
        for x in test_samples:
            test_samples_no_x = [t for t in test_samples if t != x]
            samples = [x] + random.sample(test_samples_no_x, N_ICL - 1)
            cloze_prompt = functional.make_prompt(
                template = prompt_template, 
                target = x,
                examples = samples
                )
            clozed_prompts.append(cloze_prompt)
            clozed_answers.append(x.object)
        
        outputs_lm = functional.predict_next_token(mt=mt, prompt=clozed_prompts)
        preds_lm =  [[x.token for x in xs] for xs in outputs_lm]
        recall_lm = metrics.recall(preds_lm, clozed_answers)
        reg_correct = 0
        no_bias_correct = 0
        first_token_correct = 0
        lm_correct = 0
        for i, sample, objs, prompt, preds in \
        zip(range(0,50), test_samples, clozed_answers, clozed_prompts, preds_lm):
            if (metrics.any_is_nontrivial_prefix(predictions=preds, targets=objs)):
                
                # # print(prompt)
                #uses the regular LRE
                reg_hs = build.get_hidden_state(mt, prompt, sample.subject, start)
                reg_hs = reg_hs.to(dtype=torch.float16)
                
                reg_hsweight = reg_hs.mm(reg_weight.t())
                reg_object_hs = reg_hsweight * beta + reg_bias
                reg_preds = build.get_object(mt, reg_object_hs)[0]
                
                no_bias_hs = reg_hsweight
                no_bias_preds = build.get_object(mt, no_bias_hs)[0]

                # no_weight_hs = beta * reg_hs + reg_bias
                # no_weight_preds = build.get_object(mt, no_weight_hs)[0]
                # init_subj_id = mt.tokenizer(sample.subject).input_ids[0]
                # init_subj_token = mt.tokenizer.convert_ids_to_tokens(init_subj_id)
                #print(init_subj_token)
                
                if(metrics.any_is_nontrivial_prefix(predictions=[reg_preds[0]], targets=objs)):
                    reg_correct += 1
    
                if(metrics.any_is_nontrivial_prefix(predictions=[no_bias_preds[0]], targets=objs)):
                    no_bias_correct += 1
            
                # if(metrics.any_is_nontrivial_prefix(predictions=[no_weight_preds[0]], targets=objs)):
                #     first_token_correct += 1
                    
                lm_correct += 1
    
                #if we want to compare specific samples from each relation..
                if(i < VIEW_SAMPLES):
                    print(f'{sample.subject} {reg_preds[0:3]} {no_bias_preds[0:3]}')
                    pass
        print(f'{beta},{relation.name},{reg_correct},{no_bias_correct},{lm_correct}')
        #print(f'beta,{beta},{relation.name},reg_correct,{reg_correct},no_bias_correct,{no_bias_correct},lm_correct,{lm_correct}')

using 8 samples out of 8 to build LRE
dollar ['p', 'd', 'pen'] ['kil', 'Kil', 'cent']
gun ['d', 'p', 't'] ['archivi', 'ungsseite', 'itmap']
womb ['d', 'p', 't'] ['Mam', 'd', 'ówn']
car ['d', 'p', 't'] ['A', 'тки', 'car']
harbor ['d', 'p', 't'] ['dock', 'ships', 'naval']
1,meronyms - part,0,4,47
using 8 samples out of 8 to build LRE
telephone ['head', 't', 'handle'] ['textt', 'archivi', 'Tele']
day ['p', 'd', 'pen'] ['day', 'Calendar', 'p']
car ['t', 'car', 'd'] ['A', 'чай', 'car']
typewriter ['writing', 'pen', 'p'] ['archivi', 'textt', 'typing']
bird ['p', 'pen', 'c'] ['bird', 'fe', 'estellt']
7,meronyms - part,7,6,47
using 8 samples out of 8 to build LRE
dollar ['p', 'pen', 'cent'] ['kil', 'cent', 'Kil']
tonne ['t', 'p', 'dozen'] ['ipage', 'ách', 'dozen']
jail ['d', 't', 'j'] ['prison', 'ówn', 'ben']
pie ['ungsseite', 'archivi', 'kn'] ['archivi', 'ungsseite', 'itmap']
14,meronyms - part,11,5,48
using 8 samples out of 8 to build LRE
filename ['paper', 'slot', 'wire'] ['archivi', 'textt

In [7]:
from baukit.baukit import TraceDict, parameter_names, get_parameter

In [20]:
#there are 42 layers to Gemma, each is composed of (self_attn, mlp, input, pre_attn_ln, input_ln, post_attn_ln, post_ff_ln)
model.model.layers[10]
##10 --> 42 

Gemma2DecoderLayer(
  (self_attn): Gemma2SdpaAttention(
    (q_proj): Linear(in_features=3584, out_features=4096, bias=False)
    (k_proj): Linear(in_features=3584, out_features=2048, bias=False)
    (v_proj): Linear(in_features=3584, out_features=2048, bias=False)
    (o_proj): Linear(in_features=4096, out_features=3584, bias=False)
    (rotary_emb): Gemma2RotaryEmbedding()
  )
  (mlp): Gemma2MLP(
    (gate_proj): Linear(in_features=3584, out_features=14336, bias=False)
    (up_proj): Linear(in_features=3584, out_features=14336, bias=False)
    (down_proj): Linear(in_features=14336, out_features=3584, bias=False)
    (act_fn): PytorchGELUTanh()
  )
  (input_layernorm): Gemma2RMSNorm()
  (post_attention_layernorm): Gemma2RMSNorm()
  (pre_feedforward_layernorm): Gemma2RMSNorm()
  (post_feedforward_layernorm): Gemma2RMSNorm()
)